# core

> Functions to go a fastq file to a list of genotypes

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
from fastcore.basics import *
from Bio import SeqIO
import gzip as gz
import os
from collections import defaultdict, Counter
import numpy as np
import itertools
import click
import csv
from dgrec.utils import get_mutations, mut_to_str

In [ ]:
#| hide
from dgrec.example_data import get_example_data_dir

In [ ]:
#| hide
data_path=get_example_data_dir()
os.listdir(data_path)

['paired_example1_R2.fastq.gz',
 'sacB_genotypes.csv',
 'sacB_ref.fasta',
 '__pycache__',
 'model_mms_2024_02_14.pickle',
 'sacB_example.fastq.gz',
 'paired_example1_R1.fastq.gz',
 '__init__.py']

In [ ]:
#| export

def get_UMI_genotype(fastq_path: str, #path to the input fastq file
                     ref_seq: str, #sequence of the reference amplicon
                     umi_size: int = 10, #number of nucleotides at the begining of the read that will be used as the UMI
                     quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality
                     ignore_pos: list = [], #list of positions that are ignored in the genotype
                     ) -> dict:
    
    """Takes as input a fastq_file of single read amplicon sequencing, and a reference amplicon sequence.
       Returns a dictionnary containing as keys UMIs and as values a Counter of all genotype strings read for that UMI.
    """
    with gz.open(fastq_path,'rt') as handle:
        reads=SeqIO.parse(handle,"fastq")
        n_reads=0
        n_reads_pass_Qfilter=0
        n_reads_aligned=0
        UMI_dict=defaultdict(list,{})
        for r in reads:
            n_reads+=1
            meanScore=np.mean(r.letter_annotations['phred_quality'])

            if meanScore>quality_threshold:
                n_reads_pass_Qfilter+=1
                umi=str(r.seq[:umi_size])
                mutations=get_mutations(ref_seq,r.seq[umi_size:])
                if ignore_pos:
                    mutations = [m for m in mutations if m[1] not in ignore_pos]
                n_mut=len(mutations)
                if n_mut<15: #more than 10 mutation is almost certainly crap
                    n_reads_aligned+=1
                    UMI_dict[umi].append(mut_to_str(mutations))
    
    log='n reads:\t{}\nn_reads pass filter:\t{}\nn_reads aligned:\t{}\n'.format(n_reads,n_reads_pass_Qfilter,n_reads_aligned)
    log+=f"Number of UMIs: {len(UMI_dict)}\n"
    
    UMI_gencounter={}
    umi_readcounts=[]
    for umi in UMI_dict:
        umi_readcounts.append(len(UMI_dict[umi]))
        UMI_gencounter[umi]=Counter(UMI_dict[umi])

    log+=f"Median number of reads per UMI: {np.median(umi_readcounts)}"
    print(log)
    return UMI_gencounter

In [ ]:
fastq_file="sacB_example.fastq.gz"
fastq_path=os.path.join(data_path,fastq_file)

read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq=str(ref.seq)

UMI_gencounter = get_UMI_genotype(fastq_path, ref_seq, ignore_pos=[0,1,2,138,139,140,141])

for umi in itertools.islice(UMI_gencounter,20):
    print(umi, list(UMI_gencounter[umi].items()))

n reads:	1000
n_reads pass filter:	847
n_reads aligned:	824
Number of UMIs: 814
Median number of reads per UMI: 1.0
GCATANCTCA [('A61G,-63T,A76T,A91T', 1)]
CGCATNTATA [('', 1)]
CCTTGNAGTA [('', 1)]
GGCGCNAGAA [('', 1)]
TCTCTTGTGA [('', 1)]
ATTACAGAAT [('', 1)]
CTTTTACTAT [('', 1)]
TCAAAGTTTT [('A79T,A91G', 1)]
TTAGCTCATA [('', 1)]
TCATAATGTA [('', 1)]
ATGTGCGGAT [('', 1)]
TGTGTTTATA [('', 1)]
CCATACATCC [('', 1)]
AGGGACGTTT [('A61G,A72G,A76G,A79T', 1)]
GTGTAATAGC [('', 1)]
ATGTCTTTTA [('', 1)]
TATCGGTAGT [('', 1)]
GTCGGGGGGG [('', 1)]
AAGTGGCACA [('', 1)]
AATAGAACCT [('T108A,G127T,G132T', 1)]


In [ ]:
#| export

def correct_UMI_genotypes(UMI_gencounter: dict, #the output of the get_UMI_genotype function
                          reads_per_umi_thr=2 #only assign a genotype to a UMI if we have reads_per_umi_thr reads for that genotype or more
                          ) -> dict:
    """Keeps only the genotype with the most reads for each UMI.
    Returns a dictionary with UMIs as keys and a tuple as value: (genotype string, number of reads)
    """
    UMI_gen_dict={}
    for umi in UMI_gencounter:
        gen, n =UMI_gencounter[umi].most_common(1)[0]
        if n>=reads_per_umi_thr: #only assign a genotype to a UMI if we have reads_per_umi_thr reads for that genotype or more
            UMI_gen_dict[umi]=gen

    return UMI_gen_dict

In [ ]:
correct_UMI_genotypes(UMI_gencounter)

{'CTCCGGGGAG': '',
 'TGCTTGAGTG': 'A79T',
 'AGGGCGGGCT': '',
 'ATTTCTGTTT': '',
 'TGGGGGGGCT': '',
 'GATTGGTAGA': '',
 'GAACTCTAGT': '',
 'TAACTAATCG': 'A79G,A86G,A91G'}

In [ ]:
#| export

def genotype_UMI_counter(UMI_gen_dict):
    """Takes as input the output of correct_UMI_genotypes() and 
    returns a list of genotypes sorted by the number of UMIs detected corresponding that each genotype."""
    umi_counter=Counter(UMI_gen_dict.values())
    gen_sorted_list=sorted(list(umi_counter.items()),key=lambda x: x[1], reverse=True)
    return gen_sorted_list


In [ ]:
UMI_gen_dict=correct_UMI_genotypes(UMI_gencounter, reads_per_umi_thr=0)
gen_list = genotype_UMI_counter(UMI_gen_dict)
for g in gen_list[:20]:
    print(f"{g[1]}\t{g[0]}")

675	
3	C56A
3	A76G
3	A91G
3	A91T
2	C69T
2	T122A
2	A91C
2	A105G
2	C116A
2	T60A
2	T59A
2	A68G
2	T134A
1	A61G,-63T,A76T,A91T
1	A79T,A91G
1	A61G,A72G,A76G,A79T
1	T108A,G127T,G132T
1	A48T,A86G
1	A61T,A68T,A72G,A79C,A91G


In [ ]:
#| export

def get_genotypes(fastq_path: str, #path to the input fastq file
                    ref_seq: str, #sequence of the reference amplicon
                    umi_size: int = 10, #number of nucleotides at the begining of the read that will be used as the UMI
                    quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality
                    ignore_pos: list = [], #list of positions that are ignored in the genotype
                    reads_per_umi_thr: int = 0, #minimum number of reads required to take a UMI into account. Using a number >2 enables to perform error correction for UMIs with multiple reads.
                    save_umi_data: str = None, #path to the csv file where to save the details of the genotypes reads for each UMI. If None the data isn't saved.
                    ):
    """Putting things together in a single wrapper function that takes the fastq as input and returns the list of genotypes."""
    UMI_dict = get_UMI_genotype(fastq_path, ref_seq, umi_size, quality_threshold, ignore_pos)
    if save_umi_data:
        with open(save_umi_data,"w", newline='') as handle: 
            csv_writer = csv.writer(handle,delimiter="\t",doublequote=False)
            for umi in itertools.islice(UMI_dict,20):
                csv_writer.writerow([umi,list(UMI_dict[umi].items())])

    UMI_gen_dict=correct_UMI_genotypes(UMI_dict, reads_per_umi_thr)
    gen_list = genotype_UMI_counter(UMI_gen_dict)
    print("Number of genotypes:", len(gen_list))
    return gen_list
    

In [ ]:
fastq_file="sacB_example.fastq.gz"
fastq_path=os.path.join(data_path,fastq_file)
read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq=str(ref.seq)
gen_list = get_genotypes(fastq_path, ref_seq, ignore_pos=[0,1,2,138,139,140,141], save_umi_data="test.csv")
for g in gen_list[:20]:
    print(f"{g[1]}\t{g[0]}")

n reads:	1000
n_reads pass filter:	847
n_reads aligned:	824
Number of UMIs: 814
Median number of reads per UMI: 1.0
Number of genotypes: 123
675	
3	C56A
3	A76G
3	A91G
3	A91T
2	C69T
2	T122A
2	A91C
2	A105G
2	C116A
2	T60A
2	T59A
2	A68G
2	T134A
1	A61G,-63T,A76T,A91T
1	A79T,A91G
1	A61G,A72G,A76G,A79T
1	T108A,G127T,G132T
1	A48T,A86G
1	A61T,A68T,A72G,A79C,A91G


In [ ]:
#| export

def get_UMI_genotype_paired(fastq_path_fwd: str, #path to the input fastq file reading the ref_seq in the forward orientation
                            fastq_path_rev: str, #path to the input fastq file reading the ref_seq in the reverse orientation
                            ref_seq: str, #sequence of the reference amplicon
                            fwd_span: tuple, #span of the ref_seq that is read in the forward orientation format: (start, end)
                            rev_span: tuple, #span of the ref_seq that is read in the reverse orientation format: (start, end)
                            require_perfect_pair_agreement: bool = True, #if True only pairs of reads that perfectly agree on the sequence within the common span will be used. If False the fwd sequence will be used. Will be set to False by default if there is no overlap.
                            umi_size_fwd: int = 10, #number of nucleotides at the begining of the fwd read that will be used as the UMI
                            umi_size_rev: int = 0, #number of nucleotides at the begining of the rev read that will be used as the UMI (if both are provided the umi will be the concatenation of both)
                            quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality. Both reads have to pass the threshold.
                            ignore_pos: list = [], #list of positions that are ignored in the genotype
                            N = None, #number of reads to consider (useful to get a quick view of the data without going through the whole fastq files). If None the whole data will be used.
                            ) -> dict:
    

    fwd_span = sorted(fwd_span)
    rev_span = sorted(rev_span)

    if fwd_span[1]>rev_span[0] and (rev_span[1]-rev_span[0])>0:
        overlap=True
        overlap_size=fwd_span[1]-rev_span[0]
    else:
        overlap=False
        require_perfect_pair_agreement=False

    with gz.open(fastq_path_fwd,'rt') as handle1, gz.open(fastq_path_rev,'rt') as handle2:
        fwd_reads=SeqIO.parse(handle1,"fastq")
        rev_reads=SeqIO.parse(handle2,"fastq")
        n_reads=0
        n_reads_pass_Qfilter=0
        n_reads_aligned=0
        n_reads_agree=0
        UMI_dict=defaultdict(list,{})

        paired_iter=zip(fwd_reads,rev_reads)
        if n_reads!=None:
            paired_iter=itertools.islice(paired_iter,0,N)

        for r1, r2 in paired_iter:
            n_reads+=1
            meanScore_r1=np.mean(r1.letter_annotations['phred_quality'])
            meanScore_r2=np.mean(r2.letter_annotations['phred_quality'])

            if meanScore_r1>quality_threshold and meanScore_r2>quality_threshold:
                n_reads_pass_Qfilter+=1
                umi1=str(r1.seq[:umi_size_fwd])
                umi2=str(r2.seq[:umi_size_rev])
                umi=umi1+umi2

                fwd_seq=r1.seq[umi_size_fwd:]
                rev_seq=r2.seq[umi_size_rev:].reverse_complement()

                if overlap:
                    fwd_common_seq=str(fwd_seq[rev_span[0]:fwd_span[1]])
                    rev_common_seq=str(rev_seq[:overlap_size])
                    if require_perfect_pair_agreement:
                        if fwd_common_seq==rev_common_seq:
                            n_reads_agree+=1
                        else:
                            continue
                        
                    consensus=fwd_seq+rev_seq[overlap_size:]
                    mutations=get_mutations(ref_seq[fwd_span[0]:rev_span[1]],consensus)
                else:
                    consensus=""
                    if fwd_span[1]-fwd_span[0]>0:
                        consensus+=fwd_seq
                    if rev_span[1]-rev_span[0]>0:
                        consensus+=rev_seq
                        
                    mutations=get_mutations(ref_seq[fwd_span[0]:fwd_span[1]]+ref_seq[rev_span[0]:rev_span[1]],consensus)

                if ignore_pos:
                    mutations = [m for m in mutations if m[1] not in ignore_pos]
                n_mut=len(mutations)
                if n_mut<15: #more than 10 mutation is almost certainly crap
                    n_reads_aligned+=1
                    UMI_dict[umi].append(mut_to_str(mutations))
    
    log='n reads:\t{}\nn_reads pass filter:\t{}\nn_reads aligned:\t{}\nn_pairs agree:\t{}\n'.format(n_reads,n_reads_pass_Qfilter,n_reads_aligned,n_reads_agree)
    log+=f"Number of UMIs: {len(UMI_dict)}\n"
    
    UMI_gencounter={}
    umi_readcounts=[]
    for umi in UMI_dict:
        umi_readcounts.append(len(UMI_dict[umi]))
        UMI_gencounter[umi]=Counter(UMI_dict[umi])

    log+=f"Median number of reads per UMI: {np.median(umi_readcounts)}"
    print(log)
    return UMI_gencounter

In [ ]:
fastq_path_fwd=os.path.join(data_path,"paired_example1_R2.fastq.gz")
fastq_path_rev=os.path.join(data_path,"paired_example1_R1.fastq.gz")


read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq="GAGGAGACGGTGACCTGGGTCCCCTGGCCCCAGTTGTTGAAGTCATACCCTTAGGCACCATAGTATCCAGACGCACAGTAGTACAAGGCAGTGTCCTCAGGTTTCAGGCTGTTCATTTGCAGATACACCGTGTTCTTGGCGTTGTCTTGGG"

UMI_gencounter = get_UMI_genotype_paired(fastq_path_fwd, 
                                         fastq_path_rev, 
                                         ref_seq, 
                                         fwd_span=(0,150), 
                                         rev_span=(0,0),
                                         umi_size_fwd=0,
                                         umi_size_rev=10,
                                         N=10,
                                         ignore_pos=[0,1,2,150,151])

for umi in itertools.islice(UMI_gencounter,20):
    print(umi, list(UMI_gencounter[umi].items()))

n reads:	10
n_reads pass filter:	10
n_reads aligned:	10
n_pairs agree:	0
Number of UMIs: 10
Median number of reads per UMI: 1.0
GGACGCGATA [('T50A,T51G', 1)]
GCTGTATGTT [('T50A,T51G', 1)]
GGGAAGGCGT [('', 1)]
GGCGACAGTG [('', 1)]
CCATGTCGGG [('', 1)]
CNGATGTTTG [('T33G,T34A,T51G', 1)]
GTCACACCAC [('T33C,T34C,T36C,T50A', 1)]
CCAGACTGTT [('', 1)]
CAGGGTTTAT [('T50A,T51G', 1)]
ATGGGTACGG [('T130A', 1)]


In [ ]:
#|hide
from Bio.Seq import Seq

In [ ]:
#|hide
fastq_path_fwd=os.path.join(data_path,"paired_example1_R1.fastq.gz")
fastq_path_rev=os.path.join(data_path,"paired_example1_R2.fastq.gz")


read_ref_file="sacB_ref.fasta"
ref=next(SeqIO.parse(os.path.join(data_path,read_ref_file),"fasta"))
ref_seq=str(Seq("GAGGAGACGGTGACCTGGGTCCCCTGGCCCCAGTTGTTGAAGTCATACCCTTAGGCACCATAGTATCCAGACGCACAGTAGTACAAGGCAGTGTCCTCAGGTTTCAGGCTGTTCATTTGCAGATACACCGTGTTCTTGGCGTTGTCTTGGG").reverse_complement())

UMI_gencounter = get_UMI_genotype_paired(fastq_path_fwd, 
                                         fastq_path_rev, 
                                         ref_seq, 
                                         fwd_span=(0,0), 
                                         rev_span=(0,150),
                                         umi_size_fwd=10,
                                         umi_size_rev=0,
                                         N=10,
                                         ignore_pos=[0,1,2,150,151])

for umi in itertools.islice(UMI_gencounter,20):
    print(umi, list(UMI_gencounter[umi].items()))

n reads:	10
n_reads pass filter:	10
n_reads aligned:	10
n_pairs agree:	0
Number of UMIs: 10
Median number of reads per UMI: 1.0
GGACGCGATA [('A99C,A100T', 1)]
GCTGTATGTT [('A99C,A100T', 1)]
GGGAAGGCGT [('', 1)]
GGCGACAGTG [('', 1)]
CCATGTCGGG [('', 1)]
CNGATGTTTG [('A99C,A116T,A117C', 1)]
GTCACACCAC [('A100T,A114G,A116G,A117G', 1)]
CCAGACTGTT [('', 1)]
CAGGGTTTAT [('A99C,A100T', 1)]
ATGGGTACGG [('A20T', 1)]


In [ ]:
#| export

def get_genotypes_paired(fastq_path_fwd: str, #path to the input fastq file reading the ref_seq in the forward orientation
                        fastq_path_rev: str, #path to the input fastq file reading the ref_seq in the reverse orientation
                        ref_seq: str, #sequence of the reference amplicon
                        fwd_span: tuple, #span of the ref_seq that is read in the forward orientation format: (start, end)
                        rev_span: tuple, #span of the ref_seq that is read in the reverse orientation format: (start, end)
                        require_perfect_pair_agreement: bool = True, #if True only pairs of reads that perfectly agree on the sequence within the common span will be used. If False the fwd sequence will be used. Will be set to False by default if there is no overlap.
                        umi_size_fwd: int = 10, #number of nucleotides at the begining of the fwd read that will be used as the UMI
                        umi_size_rev: int = 0, #number of nucleotides at the begining of the rev read that will be used as the UMI (if both are provided the umi will be the concatenation of both)
                        quality_threshold: int = 30, #threshold value used to filter out reads of poor average quality
                        ignore_pos: list = [], #list of positions that are ignored in the genotype
                        reads_per_umi_thr: int = 0, #minimum number of reads required to take a UMI into account. Using a number >2 enables to perform error correction for UMIs with multiple reads.
                        save_umi_data: str = None, #path to the csv file where to save the details of the genotypes reads for each UMI. If None the data isn't saved.
                        N = None, #number of reads to consider (useful to get a quick view of the data without going through the whole fastq files). If None the whole data will be used.
                        ):
    """Putting things together in a single wrapper function that takes the fastq as input and returns the list of genotypes."""
    UMI_dict = get_UMI_genotype_paired(fastq_path_fwd, 
                                         fastq_path_rev, 
                                         ref_seq, 
                                         fwd_span=fwd_span, 
                                         rev_span=rev_span,
                                         require_perfect_pair_agreement=require_perfect_pair_agreement,
                                         umi_size_fwd=umi_size_fwd,
                                         umi_size_rev=umi_size_rev,
                                         quality_threshold=quality_threshold,
                                         ignore_pos=ignore_pos,
                                         N=N
                                         )
    if save_umi_data:
        with open(save_umi_data,"w", newline='') as handle: 
            csv_writer = csv.writer(handle,delimiter="\t",doublequote=False)
            for umi in itertools.islice(UMI_dict,20):
                csv_writer.writerow([umi,list(UMI_dict[umi].items())])

    UMI_gen_dict=correct_UMI_genotypes(UMI_dict, reads_per_umi_thr)
    gen_list = genotype_UMI_counter(UMI_gen_dict)
    print("Number of genotypes:", len(gen_list))
    return gen_list

In [ ]:
gen_list = get_genotypes_paired(fastq_path_fwd, 
                        fastq_path_rev, 
                        ref_seq, 
                        fwd_span=(0,150), 
                        rev_span=(0,0),
                        umi_size_fwd=0,
                        umi_size_rev=10,
                        ignore_pos=[0,1,150,151],
                        N=10000,
                        save_umi_data="test.csv")
for g in gen_list[:20]:
    print(f"{g[1]}\t{g[0]}")

n reads:	1000
n_reads pass filter:	966
n_reads aligned:	0
n_pairs agree:	0
Number of UMIs: 0
Median number of reads per UMI: nan
Number of genotypes: 0


/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#| export
#Commande line interface
@click.command()
@click.argument('fastq', type=click.Path(exists=True))
@click.argument('ref', type=click.Path(exists=True))
@click.option('--umi_size', '-u', default=10, help="Number of nucleotides at the begining of the read that will be used as the UMI")
@click.option('--quality_threshold', '-q', default=10, help="threshold value used to filter out reads of poor average quality")
@click.option('--ignore_pos', '-i', default=[], multiple=True, help="list of positions that are ignored in the genotype, e.g. [0,1,149,150]")
@click.option('--reads_per_umi_thr', '-r', default=0, help="minimum number of reads required to take a UMI into account. Using a number >2 enables to perform error correction for UMIs with multiple reads")
@click.option('--save_umi_data','-s', default=None, help="path to a csv file to save the details of the genotypes reads for each UMI. If None the data isn't saved.")
@click.option('--output', '-o', default="genotypes.csv", help="output file path")
def dgrec_genotypes(fastq, ref, umi_size, quality_threshold, ignore_pos, reads_per_umi_thr, save_umi_data, output):
    ref=next(SeqIO.parse(ref,"fasta"))
    ref_seq=str(ref.seq)
    gen_list = get_genotypes(fastq, ref_seq, 
                             umi_size=umi_size, 
                             quality_threshold=quality_threshold, 
                             ignore_pos=ignore_pos,
                             reads_per_umi_thr=reads_per_umi_thr,
                             save_umi_data=save_umi_data)
    
    with open(output,"w") as handle:
            for g,n in gen_list:
                handle.write(f"{g}\t{n}\n")

In [ ]:
#| hide
import subprocess

In [ ]:
#| hide
#Testing the cli

fastq_file="sacB_example.fastq.gz"
read_ref_file="sacB_ref.fasta"

result = subprocess.run(["dgrec_genotypes", 
                         os.path.join(data_path,fastq_file), 
                         os.path.join(data_path,read_ref_file),
                         ])
print(result.stdout)  # Print the standard output of the command
print(result.returncode)  # Get the exit code of the command


# Remove test files

# List all files in the directory
files = os.listdir()

# Iterate over the files
for file in files:
    if file.endswith(".csv"):
        
        try:
            # Delete the file
            os.remove(file)
        except PermissionError:
            print(f"Permission denied to delete file '{file}'.")
        except FileNotFoundError:
            print(f"File '{file}' not found.")

/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/Bio/__init__.py:138: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages
  warnings.warn(
Traceback (most recent call last):
  File "/home/dbikard/anaconda3/envs/DGRec/bin/dgrec_genotypes", line 33, in <module>
    sys.exit(load_entry_point('dgrec', 'console_scripts', 'dgrec_genotypes')())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dbikard/anaconda3/envs/DGRec/lib/python3.11/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^

None
1


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()